In [164]:
# Library Imports
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
from pinocchio import RobotWrapper
import numpy as np
from numpy.linalg import *
import matplotlib.pyplot as plt
plt.style.use('dark_background')

from time import sleep

In [165]:
# Read the .stl files
package_dir = '/home/kanishk/Desktop/Pinocchio-Based-Robot-Solver'
urdf_path = '/home/kanishk/Desktop/Pinocchio-Based-Robot-Solver/model/fingeredu.urdf'

In [166]:
# Build the robot 
robot = RobotWrapper.BuildFromURDF(urdf_path, package_dir)

In [167]:
# Display the visual of the robot
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()
viz.displayCollisions(True)
q0 = pin.neutral(robot.model)
viz.display(q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/
<ipython-input-167-6974781be780>:5: UserWarning: Plotting collision meshes is not available for MeshcatVisualizer
  viz.displayCollisions(True)


In [168]:
# Get the FrameID of the EOAT
EOAT_ID = robot.model.getFrameId('finger_tip_link')

In [169]:
# Set Robot to Better Kinematic Pose
q = pin.randomConfiguration(robot.model)

# Computing Forward Kinematics
def compute_ForwardKinematics(robot, q):
    pin.framesForwardKinematics(robot.model, robot.data, q)
    pos = np.array(robot.data.oMf[EOAT_ID].translation)
    viz.display(pos)
    return pos

In [170]:
# Compute the Jacobian
def compute_Jacobian(robot, q):
    pin.computeJointJacobians(robot.model, robot.data, q)
    return pin.getFrameJacobian(robot.model, robot.data, EOAT_ID, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED,)[:3]

In [171]:
# Compute Inverse Kinematics
def Trajectory_Planner(robot, goal_pos, joint_pos, interpolation='linear'):
    # Initiate Callers
    q = joint_pos
    init_y = compute_ForwardKinematics(robot, q)
    y_target = goal_pos

    # Constants
    smooth = 1e-2
    W = 1e-4 * np.eye(3)
    max_steps = 10

    for i in range(1, int(max_steps)):
        y = compute_ForwardKinematics(robot, q)
        J = compute_Jacobian(robot, q) 

        # Compute Error
        Error = goal_pos - y

        if interpolation == 'joint':
            pass
        if interpolation == 'linear':
            y_target = init_y + (i/max_steps) * (goal_pos - init_y)  

        q = smooth * np.linalg.inv(J.T @ J + W) @ J.T @ (y_target - y)

        viz.display(q)
        sleep(0.2)

    return q

In [172]:
# Check Trajectory Generation

# Set the Robot in Random Position
q = pin.randomConfiguration(robot.model)
viz.display(q)

# Augment 'goal_pos' from 'curr_pos'
curr_pos = compute_ForwardKinematics(robot, q)
print (curr_pos)

[-0.01399262  0.01225668  0.1388154 ]


In [173]:
_q = Trajectory_Planner(robot, [0, 0, 0], q, interpolation='joint')
print (compute_ForwardKinematics(robot, _q))

[-9.24419534e-11 -5.89464864e-02 -3.70931337e-02]
